In [1]:
from skimage.feature import local_binary_pattern
import numpy as np
import cv2
import os

def calcula_lbp(imagem, P, R, metodo):
    # Calcular o LBP
    lbp = local_binary_pattern(imagem, P, R, metodo)
    # Calcular o histograma LBP
    n_bins = int(lbp.max() + 1)
    hist, bins = np.histogram(lbp.ravel(), bins=n_bins, range=(0, n_bins), density=True)
    return hist

def calcula_lbp_por_partes(imagem, P, R, metodo, num_partes=3):
    altura = imagem.shape[0]  # Altura da imagem
    largura = imagem.shape[1]  # Largura da imagem
    
    # Dividir a imagem horizontalmente em `num_partes` partes
    parte_altura = altura // num_partes
    
    histogramas = []
    
    for i in range(num_partes):
        # Definir a parte da imagem
        inicio = i * parte_altura
        fim = (i + 1) * parte_altura if i < num_partes - 1 else altura  # Garantir que a última parte pegue até o fim da imagem
        
        parte_imagem = imagem[inicio:fim, :]
        
        # Calcular o LBP e o histograma para a parte
        hist = calcula_lbp(parte_imagem, P, R, metodo)
        histogramas.append(hist)
    
    # Retornar os histogramas de cada parte separadamente
    return histogramas[0], histogramas[1], histogramas[2]

# Parâmetros LBP
P = 8  # Número de pontos vizinhos
R = 2  # Raio
metodo = 'nri_uniform'

# Diretórios
diretorios = ['base/fold1', 'base/fold2', 'base/fold3']

# Listas para armazenar os histogramas de cada parte
hist_topo = []
hist_meio = []
hist_base = []
tags = []

# Carregar as imagens e calcular os histogramas LBP
for i, diretorio in enumerate(diretorios):
    lista_arquivos = os.listdir(diretorio)
    for j, arquivo_img in enumerate(lista_arquivos):
        caminho_imagem = os.path.join(diretorio, arquivo_img)
        imagem = cv2.imread(caminho_imagem, cv2.IMREAD_GRAYSCALE)
        if imagem is not None:
            # Calcular os histogramas para as três partes (topo, meio, base)
            hist_top, hist_mid, hist_bot = calcula_lbp_por_partes(imagem, P, R, metodo, num_partes=3)
            hist_topo.append(hist_top)
            hist_meio.append(hist_mid)
            hist_base.append(hist_bot)
            tags.append(j // 30 + 1)  # Tag para cada imagem

# Converter as listas para arrays numpy
hist_topo = np.array(hist_topo)
hist_meio = np.array(hist_meio)
hist_base = np.array(hist_base)
tags_ori = np.array(tags)


In [2]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

# Função para realizar predições com probabilidades
def kfoldcv_proba(hist, tags, k):
    fold_size = len(hist) // k
    all_probas = []
    y_tests = []

    for i in range(k):
        # Define os índices de teste e treino baseados na ordem
        test_idx = np.arange(i*fold_size, (i+1)*fold_size)
        train_idx = np.concatenate([np.arange(0, i*fold_size), np.arange((i+1)*fold_size, len(hist))])
        
        X_train, X_test = hist[train_idx], hist[test_idx]
        y_train, y_test = tags[train_idx], tags[test_idx]
        
        # Padronizar os dados
        scaler = StandardScaler().fit(X_train)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)

        # Configurar os parâmetros para o GridSearchCV (SVM)
        param_grid = {
            'C': [0.289],
            'gamma': [1],
            'kernel': ['linear', 'rbf']
        }

        svm_model = SVC(random_state=42, probability=True)
        grid_search = GridSearchCV(estimator=svm_model, param_grid=param_grid, cv=3, scoring='accuracy')
        grid_search.fit(X_train, y_train)
        
        # Melhor modelo encontrado pelo Grid Search
        best_model = grid_search.best_estimator_

        # Fazer previsões de probabilidades
        y_proba = best_model.predict_proba(X_test)
        all_probas.append(y_proba)
        y_tests.append(y_test)
    
    return np.concatenate(all_probas), np.concatenate(y_tests)

# Realizar a validação cruzada com probabilidades para cada conjunto de dados
proba_topo_svm, y_test = kfoldcv_proba(hist_topo, tags_ori, 3)
proba_meio_svm, _ = kfoldcv_proba(hist_meio, tags_ori, 3)
proba_base_svm, _ = kfoldcv_proba(hist_base, tags_ori, 3)

# Fusão das predições usando a regra da soma
fused_proba_soma = (proba_topo_svm + proba_meio_svm + proba_base_svm) / 3
fused_pred_soma = np.argmax(fused_proba_soma, axis=1)

for i in range(0, len(fused_pred_soma)):
    fused_pred_soma[i] = fused_pred_soma[i] + 1

# Fusão das predições usando a regra do produto
fused_proba_produto = proba_topo_svm * proba_meio_svm * proba_base_svm
fused_pred_produto = np.argmax(fused_proba_produto, axis=1)

for i in range(0, len(fused_pred_produto)):
    fused_pred_produto[i] = fused_pred_produto[i] + 1

# Avaliação das fusões

def evaluate_predictions(y_true, y_pred, method_name):
    accuracy = accuracy_score(y_true, y_pred) * 100
    precision = precision_score(y_true, y_pred, average='weighted') * 100
    recall = recall_score(y_true, y_pred, average='weighted') * 100
    f1 = f1_score(y_true, y_pred, average='weighted') * 100

    print(f'{method_name} - Acurácia: {accuracy:.2f}%')
    print(f'{method_name} - Precisão: {precision:.2f}%')
    print(f'{method_name} - Recall: {recall:.2f}%')
    print(f'{method_name} - F1-score: {f1:.2f}%')

# Avaliar as predições usando a regra da soma
evaluate_predictions(y_test, fused_pred_soma, "Regra da Soma")

print('----------------------------------------------------')

# Avaliar as predições usando a regra do produto
evaluate_predictions(y_test, fused_pred_produto, "Regra do Produto")


Regra da Soma - Acurácia: 81.33%
Regra da Soma - Precisão: 81.53%
Regra da Soma - Recall: 81.33%
Regra da Soma - F1-score: 81.32%
----------------------------------------------------
Regra do Produto - Acurácia: 82.00%
Regra do Produto - Precisão: 82.12%
Regra do Produto - Recall: 82.00%
Regra do Produto - F1-score: 81.98%


In [3]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
import numpy as np

# Função para realizar predições com probabilidades usando KNN
def kfoldcv_knn_proba(hist, tags, k):
    fold_size = len(hist) // k
    all_probas = []
    y_tests = []

    for i in range(k):
        # Define os índices de teste e treino baseados na ordem
        test_idx = np.arange(i*fold_size, (i+1)*fold_size)
        train_idx = np.concatenate([np.arange(0, i*fold_size), np.arange((i+1)*fold_size, len(hist))])

        X_train, X_test = hist[train_idx], hist[test_idx]
        y_train, y_test = tags[train_idx], tags[test_idx]

        # Padronizar os dados
        scaler = StandardScaler().fit(X_train)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)

        # Definir a grade de parâmetros para o Grid Search do KNN
        param_grid = {
            'n_neighbors': [3, 5, 7, 9, 11],
            'metric': ['canberra', 'minkowski', 'euclidean', 'manhattan']
        }

        # Treinar o modelo KNN usando Grid Search
        knn_model = KNeighborsClassifier()
        grid_search = GridSearchCV(knn_model, param_grid, cv=3, scoring='accuracy', n_jobs=-1)
        grid_search.fit(X_train, y_train)

        # Melhor modelo encontrado pelo Grid Search
        best_knn = grid_search.best_estimator_

        # Fazer previsões de probabilidades
        y_proba = best_knn.predict_proba(X_test)
        all_probas.append(y_proba)
        y_tests.append(y_test)

    return np.concatenate(all_probas), np.concatenate(y_tests)

# Realizar a validação cruzada com probabilidades para cada conjunto de dados
proba_topo_knn, y_test = kfoldcv_knn_proba(hist_topo, tags_ori, 3)
proba_meio_knn, _ = kfoldcv_knn_proba(hist_meio, tags_ori, 3)
proba_base_knn, _ = kfoldcv_knn_proba(hist_base, tags_ori, 3)

# Fusão das predições usando a regra da soma
fused_proba_soma = (proba_topo_knn + proba_meio_knn + proba_base_knn) / 3
fused_pred_soma = np.argmax(fused_proba_soma, axis=1)

for i in range(0, len(fused_pred_soma)):
    fused_pred_soma[i] = fused_pred_soma[i] + 1

# Fusão das predições usando a regra do produto
fused_proba_produto = proba_topo_knn * proba_meio_knn * proba_base_knn
fused_pred_produto = np.argmax(fused_proba_produto, axis=1)

for i in range(0, len(fused_pred_produto)):
    fused_pred_produto[i] = fused_pred_produto[i] + 1

# Função para avaliar as predições
def evaluate_predictions(y_true, y_pred, method_name):
    accuracy = accuracy_score(y_true, y_pred) * 100
    precision = precision_score(y_true, y_pred, average='weighted') * 100
    recall = recall_score(y_true, y_pred, average='weighted') * 100
    f1 = f1_score(y_true, y_pred, average='weighted') * 100

    print(f'{method_name} - Acurácia: {accuracy:.2f}%')
    print(f'{method_name} - Precisão: {precision:.2f}%')
    print(f'{method_name} - Recall: {recall:.2f}%')
    print(f'{method_name} - F1-score: {f1:.2f}%')

# Avaliar as predições usando a regra da soma
evaluate_predictions(y_test, fused_pred_soma, "Regra da Soma")

print('----------------------------------------------------')

# Avaliar as predições usando a regra do produto
evaluate_predictions(y_test, fused_pred_produto, "Regra do Produto")


Regra da Soma - Acurácia: 70.67%
Regra da Soma - Precisão: 70.88%
Regra da Soma - Recall: 70.67%
Regra da Soma - F1-score: 70.33%
----------------------------------------------------
Regra do Produto - Acurácia: 68.44%
Regra do Produto - Precisão: 73.01%
Regra do Produto - Recall: 68.44%
Regra do Produto - F1-score: 69.19%


In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
import numpy as np

# Função para realizar predições com probabilidades usando RandomForest
def kfoldcv_rf_proba(hist, tags, k):
    fold_size = len(hist) // k
    all_probas = []
    y_tests = []

    for i in range(k):
        # Define os índices de teste e treino baseados na ordem
        test_idx = np.arange(i*fold_size, (i+1)*fold_size)
        train_idx = np.concatenate([np.arange(0, i*fold_size), np.arange((i+1)*fold_size, len(hist))])

        X_train, X_test = hist[train_idx], hist[test_idx]
        y_train, y_test = tags[train_idx], tags[test_idx]

        # Padronizar os dados
        scaler = StandardScaler().fit(X_train)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)

        # Treinar o modelo Random Forest
        rf_model = RandomForestClassifier(n_estimators=4000, random_state=0, n_jobs=-1)
        rf_model.fit(X_train, y_train)

        # Fazer previsões de probabilidades
        y_proba = rf_model.predict_proba(X_test)
        all_probas.append(y_proba)
        y_tests.append(y_test)

    return np.concatenate(all_probas), np.concatenate(y_tests)

# Realizar a validação cruzada com probabilidades para cada conjunto de dados
proba_topo_rf, y_test = kfoldcv_rf_proba(hist_topo, tags_ori, 3)
proba_meio_rf, _ = kfoldcv_rf_proba(hist_meio, tags_ori, 3)
proba_base_rf, _ = kfoldcv_rf_proba(hist_base, tags_ori, 3)

# Fusão das predições usando a regra da soma
fused_proba_soma = (proba_topo_rf + proba_meio_rf + proba_base_rf) / 3
fused_pred_soma = np.argmax(fused_proba_soma, axis=1)

for i in range(0, len(fused_pred_soma)):
    fused_pred_soma[i] = fused_pred_soma[i] + 1

# Fusão das predições usando a regra do produto
fused_proba_produto = proba_topo_rf * proba_meio_rf * proba_base_rf
fused_pred_produto = np.argmax(fused_proba_produto, axis=1)

for i in range(0, len(fused_pred_produto)):
    fused_pred_produto[i] = fused_pred_produto[i] + 1

# Função para avaliar as predições
def evaluate_predictions(y_true, y_pred, method_name):
    accuracy = accuracy_score(y_true, y_pred) * 100
    precision = precision_score(y_true, y_pred, average='weighted') * 100
    recall = recall_score(y_true, y_pred, average='weighted') * 100
    f1 = f1_score(y_true, y_pred, average='weighted') * 100

    print(f'{method_name} - Acurácia: {accuracy:.2f}%')
    print(f'{method_name} - Precisão: {precision:.2f}%')
    print(f'{method_name} - Recall: {recall:.2f}%')
    print(f'{method_name} - F1-score: {f1:.2f}%')

# Avaliar as predições usando a regra da soma
evaluate_predictions(y_test, fused_pred_soma, "Regra da Soma")

print('----------------------------------------------------')

# Avaliar as predições usando a regra do produto
evaluate_predictions(y_test, fused_pred_produto, "Regra do Produto")


Regra da Soma - Acurácia: 73.33%
Regra da Soma - Precisão: 73.38%
Regra da Soma - Recall: 73.33%
Regra da Soma - F1-score: 72.96%
----------------------------------------------------
Regra do Produto - Acurácia: 74.33%
Regra do Produto - Precisão: 74.38%
Regra do Produto - Recall: 74.33%
Regra do Produto - F1-score: 74.13%


dividida em 10 partes

In [5]:
from skimage.feature import local_binary_pattern
import numpy as np
import cv2
import os

def calcula_lbp(imagem, P, R, metodo):
    # Calcular o LBP
    lbp = local_binary_pattern(imagem, P, R, metodo)
    # Calcular o histograma LBP
    n_bins = int(lbp.max() + 1)
    hist, bins = np.histogram(lbp.ravel(), bins=n_bins, range=(0, n_bins), density=True)
    return hist

def calcula_lbp_por_partes(imagem, P, R, metodo, num_partes=10):
    altura = imagem.shape[0]  # Altura da imagem
    largura = imagem.shape[1]  # Largura da imagem
    
    # Dividir a imagem horizontalmente em `num_partes` partes
    parte_altura = altura // num_partes
    
    histogramas = []
    
    for i in range(num_partes):
        # Definir a parte da imagem
        inicio = i * parte_altura
        fim = (i + 1) * parte_altura if i < num_partes - 1 else altura  # Garantir que a última parte pegue até o fim da imagem
        
        parte_imagem = imagem[inicio:fim, :]
        
        # Calcular o LBP e o histograma para a parte
        hist = calcula_lbp(parte_imagem, P, R, metodo)
        histogramas.append(hist)
    
    # Retornar os histogramas de cada parte separadamente (agora são 10)
    return histogramas

# Parâmetros LBP
P = 8  # Número de pontos vizinhos
R = 2  # Raio
metodo = 'nri_uniform'

# Diretórios
diretorios = ['base/fold1', 'base/fold2', 'base/fold3']

# Lista para armazenar os histogramas de cada parte
hist_partes = [[] for _ in range(10)]
tags = []

# Carregar as imagens e calcular os histogramas LBP
for i, diretorio in enumerate(diretorios):
    lista_arquivos = os.listdir(diretorio)
    for j, arquivo_img in enumerate(lista_arquivos):
        caminho_imagem = os.path.join(diretorio, arquivo_img)
        imagem = cv2.imread(caminho_imagem, cv2.IMREAD_GRAYSCALE)
        if imagem is not None:
            # Calcular os histogramas para as 10 partes
            histogramas = calcula_lbp_por_partes(imagem, P, R, metodo, num_partes=10)
            for idx, hist in enumerate(histogramas):
                hist_partes[idx].append(hist)
            tags.append(j // 30 + 1)  # Tag para cada imagem

# Converter as listas para arrays numpy
hist_partes = [np.array(hist) for hist in hist_partes]
tags_ori = np.array(tags)


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (900,) + inhomogeneous part.